In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
data = pd.read_csv('../../communal/Ames_Housing_Price_Data_cleaned_2.csv')

In [4]:
data.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,PID,GrLivArea,SalePrice_x,MSSubClass,MSZoning,LotFrontage,LotArea_x,...,PA-UnTyp,PA-UntNo,Date,Source,NmbrBRs,Short_hand,Full_Name,lat,long,distToUni
0,0,0,1,909176150,856,126000,30,RL,64.9,7890,...,NaN,NaN,10-Jul-20,Ames City Assessor,2.0,SWISU,South & West of Iowa State University,NaN,NaN,inf
1,1,1,2,905476230,1049,139500,120,RL,42.0,4235,...,NaN,NaN,10-Jul-20,Ames City Assessor,2.0,Edwards,Edwards,42.023035,-93.673386,1.39
2,2,1,2,905476230,1049,139500,120,RL,42.0,4235,...,NaN,NaN,10-Jul-20,Ames City Assessor,2.0,Edwards,Edwards,42.023035,-93.673386,1.39
3,3,2,3,911128020,1001,124900,30,C (all),60.0,6060,...,NaN,NaN,10-Jul-20,Ames City Assessor,2.0,IDOTRR,Iowa DOT and Rail Road,NaN,NaN,inf
4,4,3,4,535377150,1039,114000,70,RL,80.0,8146,...,NaN,NaN,10-Jul-20,Ames City Assessor,2.0,OldTown,Old Town,42.033237,-93.617133,1.59


In [6]:
data['dBin'] = pd.cut(data['distToUni'],3)

ValueError: cannot specify integer `bins` when input data contains infinity

In [9]:
data[data['distToUni'] < 1]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,PID,GrLivArea,SalePrice_x,MSSubClass,MSZoning,LotFrontage,LotArea_x,...,PA-UnTyp,PA-UntNo,Date,Source,NmbrBRs,Short_hand,Full_Name,lat,long,distToUni
9,9,8,9,923426010,889,137500,20,RL,74.0,12395,...,NaN,NaN,10-Jul-20,Ames City Assessor,3.0,Mitchel,Mitchell,42.029834,-93.629135,0.93
12,12,11,12,923426070,1274,136000,60,RL,63.0,10475,...,NaN,NaN,10-Jul-20,Ames City Assessor,3.0,Mitchel,Mitchell,42.029834,-93.629135,0.93
20,20,19,20,923252075,1091,131250,80,RL,62.0,7706,...,NaN,NaN,10-Jul-20,Ames City Assessor,2.0,Mitchel,Mitchell,42.029834,-93.629135,0.93
34,34,33,34,903400180,1848,207000,50,RL,64.0,13053,...,NaN,NaN,10-Jul-20,Ames City Assessor,4.0,BrkSide,Brookside,42.028240,-93.629960,0.87
44,44,41,42,914452190,1080,134000,20,RL,71.0,9187,...,NaN,NaN,10-Jul-20,Ames City Assessor,3.0,Mitchel,Mitchell,42.029834,-93.629135,0.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2581,2581,2537,721,914476520,1560,201000,20,RL,129.0,9196,...,NaN,NaN,10-Jul-20,Ames City Assessor,3.0,Mitchel,Mitchell,42.029834,-93.629135,0.93
2583,2583,2539,723,903401050,1505,145000,50,RL,68.7,9144,...,NaN,NaN,10-Jul-20,Ames City Assessor,3.0,BrkSide,Brookside,42.028240,-93.629960,0.87
2585,2585,2541,725,903232090,1573,162900,50,RM,52.0,6240,...,NaN,NaN,10-Jul-20,Ames City Assessor,3.0,BrkSide,Brookside,42.028240,-93.629960,0.87
2589,2589,2545,729,903202170,1654,137000,70,RL,50.0,4960,...,NaN,NaN,10-Jul-20,Ames City Assessor,3.0,BrkSide,Brookside,42.028240,-93.629960,0.87


In [18]:
data[data['distToUni'].isna()]['Neighborhood_x'].unique()

array(['NAmes', 'Greens', 'GrnHill', 'Landmrk'], dtype=object)